In [1]:
#If you are on google collab, then install the biopython packages to this system
#The code for installing biopython comes from:
#https://colab.research.google.com/github/chris-rands/biopython-coronavirus/blob/master/biopython-coronavirus-notebook.ipynb#scrollTo=w5rnDvKdukEo
try:
    import google.colab
    # Running on Google Colab, so install Biopython first
    !pip install biopython
except ImportError:
    pass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.2 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%!
cd /content/drive/MyDrive/CMPBIO250/rec3

['/bin/bash: line 1: cd: /content/drive/MyDrive/CMPBIO250/rec3: No such file or directory']

In [4]:
#Import the biopython and system libraries
import Bio
import sys

#Import the sequence object
from Bio.Seq import Seq

#Import the module to read FASTA Sequences
from Bio import SeqIO

from urllib.request import urlretrieve

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

Python version: sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0)
Biopython version: 1.81


In [5]:
#Import pairwise alignment package
from Bio import pairwise2
from Bio.pairwise2 import format_alignment



/usr/local/lib/python3.10/dist-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [6]:
################################################
# Example 1 - Basic sequence alignment      #
################################################

#Set the sequence variables
seqStrA = "TAGATG";
seqStrB = "CCTATAGG";

#Create sequence objects
seqA = Seq(seqStrA);
seqB = Seq(seqStrB);

print("\n" + seqA + "\n")

#Perform a basic global alignment of two sequences
alignments_ab = pairwise2.align.globalxx(seqA, seqB);

#Print out the first two alignments that the function returns
print(alignments_ab[0]);
print(alignments_ab[1]);

#Print a formatted version of the alignment
print(format_alignment(*alignments_ab[0]));

#Print the score of each alignment and the number of alignments total
curAlnNum = 0;
for curAln in alignments_ab :
	#print(curAln[2])
	curAlnNum = curAlnNum + 1;

print(curAlnNum);

#Print the number of alignments returned
print(len(alignments_ab));



TAGATG

Alignment(seqA='--TAGAT-G-', seqB='CCT--ATAGG', score=4.0, start=0, end=10)
Alignment(seqA='--TAGAT-G-', seqB='CCTA--TAGG', score=4.0, start=0, end=10)
--TAGAT-G-
  |  || | 
CCT--ATAGG
  Score=4

13
13


In [7]:
################################################
# Example 2 - Adjusting parameters      #
################################################


#Adding new parameters - basic
alignments_ab2 = pairwise2.align.globalms(seqA, seqB, 1, 0, 0, 0);
print(format_alignment(*alignments_ab2[0]))

#Adding new parameters - gap penalty
alignments_ab3 = pairwise2.align.globalms(seqA, seqB, 1, 0, -5, -1);
print(format_alignment(*alignments_ab3[0]))

#Adding new parameters - mismatch penalty
alignments_ab4 = pairwise2.align.globalms(seqA, seqB, 1, -5, 0, 0);
print(format_alignment(*alignments_ab4[0]))

#Adding new parameters - mismatch and gap penalty
alignments_ab5 = pairwise2.align.globalms(seqA, seqB, 2, -1, -3, -.5);
print(format_alignment(*alignments_ab5[0]))


--TAGAT-G-
  |  || | 
CCT--ATAGG
  Score=4

--TAGATG
  ||.|.|
CCTATAGG
  Score=-2

--TAGAT-G-
  |  || | 
CCT--ATAGG
  Score=4

--TAGATG
  ||.|.|
CCTATAGG
  Score=2.5



In [8]:
################################################
# Example 3 - Try performing a local alignment on the sequences      #
################################################

#Perform a local alignment of the sequences a and b.
#How does this alignment differ from the global alignment?

test_align1 = pairwise2.align.localxx(seqA, seqB)
print(format_alignment(*test_align1[0]))

#Perform a local alignment of the sequences a and b.
#Add a gap penalty of 5 and an extension penalty of 1.
#How does this alignment differ from the global alignment?

test_align2 = pairwise2.align.localmd(seqA, seqB,1,0,-5,-1,0,0)
print(format_alignment(*test_align2[0]))

#Perform a local alignment of the sequences a and b.
#Add a mismatch penalty of 5. Do not include a gap penalty
#How does this alignment differ from the global alignment?

test_align3 = pairwise2.align.localmx(seqA, seqB,1,-5)
print(format_alignment(*test_align3[0]))

#Perform a local alignment of the sequences a and b.
#Add a mismatch penalty of 1. Add a gap penalty of 3 and an extension penalty of 0.5.
#How does this alignment differ from the global alignment?

test_align4 = pairwise2.align.localmd(seqA, seqB,1,-1,-3,-0.5,0,0)
print(format_alignment(*test_align4[0]))

1 TAGAT--G
  |  ||  |
3 T--ATAGG
  Score=4

1 TAGATG
  ||.|.|
3 TATAGG
  Score=4

1 TAGAT--G
  |  ||  |
3 T--ATAGG
  Score=4

1 TAGATG
  |||  |
5 TAG--G
  Score=4



In [9]:
################################################
# Example 4 - Read in and align a sequence      #
################################################

############ Read in the mystery sequence #################

mysterySeqFn = "http://www.andrew.cmu.edu/user/apfennin/2021_02250/mystery_mrna.fa";
localInputFn = "mystery_mrna.fa"
urlretrieve(mysterySeqFn, localInputFn)

#Read in the fasta sequence and store it as a sequence object
mysterySeqR = SeqIO.read(localInputFn, "fasta")

#Store the fasta sequence as a sequence object
mysterySeq = mysterySeqR.seq;

#Print the sequence
print(str(mysterySeq));

#Set the sequence molecule type as "DNA"
mysterySeq.molecule_type = "DNA";

#Print the sequence molecule type
print(mysterySeq.molecule_type);


ATTCATAAAACGCTTGTTATAAAAGCAGTGGCTGCGGCGCCTCGTACTCCAACCGCATCTGCAGCGAGCATCTGAGAAGCCAAGACTGAGCCGGCGGCCGCGGCGCAGCGAACGAGCAGTGACCGTGCTCCTACCCAGCTCTGCTCCACAGCGCCCACCTGTCTCCGCCCCTCGGCCCCTCGCCCGGCTTTGCCTAACCGCCACGATGATGTTCTCGGGCTTCAACGCAGACTACGAGGCGTCATCCTCCCGCTGCAGCAGCGCGTCCCCGGCCGGGGATAGCCTCTCTTACTACCACTCACCCGCAGACTCCTTCTCCAGCATGGGCTCGCCTGTCAACGCGCAGGACTTCTGCACGGACCTGGCCGTCTCCAGTGCCAACTTCATTCCCACGGTCACTGCCATCTCGACCAGTCCGGACCTGCAGTGGCTGGTGCAGCCCGCCCTCGTCTCCTCCGTGGCCCCATCGCAGACCAGAGCCCCTCACCCTTTCGGAGTCCCCGCCCCCTCCGCTGGGGCTTACTCCAGGGCTGGCGTTGTGAAGACCATGACAGGAGGCCGAGCGCAGAGCATTGGCAGGAGGGGCAAGGTGGAACAGTTATCTCCAGAAGAAGAAGAGAAAAGGAGAATCCGAAGGGAAAGGAATAAGATGGCTGCAGCCAAATGCCGCAACCGGAGGAGGGAGCTGACTGATACACTCCAAGCGGAGACAGACCAACTAGAAGATGAGAAGTCTGCTTTGCAGACCGAGATTGCCAACCTGCTGAAGGAGAAGGAAAAACTAGAGTTCATCCTGGCAGCTCACCGACCTGCCTGCAAGATCCCTGATGACCTGGGCTTCCCAGAAGAGATGTCTGTGGCTTCCCTTGATCTGACTGGGGGCCTGCCAGAGGTTGCCACCCCGGAGTCTGAGGAGGCCTTCACCCTGCCTCTCCTCAATGACCCTGAGCCCAAGCCCTCAGTGGAACCTGTCAAGAGCATCAGCAGCATGGAGCTGAAG

In [10]:
############ Read in the CREB sequence #################

#The URL of the CREB file.
crebSeqFn = "http://www.andrew.cmu.edu/user/apfennin/2021_02250/creb_mrna.fa";

#The local file you want to copy it to.
localCrebFn = "creb_mrna.fa"

#Copy the file from the internet to your current collab session
urlretrieve(crebSeqFn, localCrebFn)

#Read in the fasta sequence and store it as a sequence object
crebSeqR = SeqIO.read(localCrebFn, "fasta")

#Store the fasta sequence as a sequence object
crebSeq = crebSeqR.seq;

#Print the sequence
print(str(crebSeq));

#Set the sequence molecule type as "DNA"
crebSeq.molecule_type = "DNA";

#Print the sequence molecule type
print(crebSeq.molecule_type);

TGTTTCCGTGCGCGGCCGCTGCGCACTCGGCACTGGGCGGCGCTGGCTGGCTCCCTGGCTGCGGCTCCTCAGTCGGCGGCGGCTGCTGCTGCCTGTGGCCCGGGCGGCTGGGAGAAGCGGAGTGTTGGTGAGTGACGCGGCGGAGGTGTAGTTTGACGCGGTGTGTTACGTGGGGGAGAGAATAAAACTCCAGCGAGATCCGGGCCGTGAACGAAAGCAGTGACGGAGGAGCTTGTACCACCGGTAACTAAATGACCATGGAATCTGGAGCCGAGAACCAGCAGAGTGGAGATGCAGCTGTAACAGAAGCTGAAAACCAACAAATGACAGTTCAAGCCCAGCCACAGATTGCCACATTAGCCCAGGTATCTATGCCAGCAGCTCATGCAACATCATCTGCTCCCACCGTAACTCTAGTACAGCTGCCCAATGGGCAGACAGTTCAAGTCCATGGAGTCATTCAGGCGGCCCAGCCATCAGTTATTCAGTCTCCACAAGTCCAAACAGTTCAGATTTCAACTATTGCAGAAAGTGAAGATTCACAGGAGTCAGTGGATAGTGTAACTGATTCCCAAAAGCGAAGGGAAATTCTTTCAAGGAGGCCTTCCTACAGGAAAATTTTGAATGACTTATCTTCTGATGCACCAGGAGTGCCAAGGATTGAAGAAGAGAAGTCTGAAGAGGAGACTTCAGCACCTGCCATCACCACTGTAACGGTGCCAACTCCAATTTACCAAACTAGCAGTGGACAGTATATTGCCATTACCCAGGGAGGAGCAATACAGCTGGCTAACAATGGTACCGATGGGGTACAGGGCCTGCAAACATTAACCATGACCAATGCAGCAGCCACTCAGCCGGGTACTACCATTCTACAGTATGCACAGACCACTGATGGACAGCAGATCTTAGTGCCCAGCAACCAAGTTGTTGTTCAAGGTACTCAAAAATTGTAAAGCAGGATGTCAGTGAATTTGAATTCTGAACGTCAGTTTGAAGA